## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-14-18-52-56 +0000,nypost,"Hamas will disarm or ‘we will disarm them,’ Tr...",https://nypost.com/2025/10/14/us-news/hamas-wi...
1,2025-10-14-18-52-47 +0000,nyt,"Israel Pressures Hamas to Return Bodies, but G...",https://www.nytimes.com/2025/10/14/world/middl...
2,2025-10-14-18-51-04 +0000,nyt,U.S. Military Kills 6 People in Another Attack...,https://www.nytimes.com/2025/10/14/us/politics...
3,2025-10-14-18-50-47 +0000,nyt,Madagascar’s President Goes Into Hiding,https://www.nytimes.com/2025/10/13/world/afric...
4,2025-10-14-18-38-00 +0000,wsj,Fed Chair Powell left the central bank on trac...,https://www.wsj.com/economy/central-banking/po...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,51
13,gaza,24
700,china,17
0,hamas,16
147,new,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
330,2025-10-13-20-45-44 +0000,nypost,Dems largely quiet about Trump’s role in histo...,https://nypost.com/2025/10/13/us-news/top-dems...,127
127,2025-10-14-14-24-22 +0000,nypost,Kamala Harris refuses to name Trump as she lau...,https://nypost.com/2025/10/14/us-news/kamala-h...,127
325,2025-10-13-20-53-37 +0000,wapo,Trump and world leaders meet in Egypt on futur...,https://www.washingtonpost.com/world/2025/10/1...,122
103,2025-10-14-15-25-15 +0000,nypost,Hamas won’t give up arms or Gaza control — des...,https://nypost.com/2025/10/14/world-news/hamas...,121
49,2025-10-14-17-30-22 +0000,nypost,"Gaza peace process ‘IS NOT DONE,’ Trump warns ...",https://nypost.com/2025/10/14/us-news/gaza-pea...,120


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
330,127,2025-10-13-20-45-44 +0000,nypost,Dems largely quiet about Trump’s role in histo...,https://nypost.com/2025/10/13/us-news/top-dems...
302,60,2025-10-13-21-59-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...
260,58,2025-10-14-01-59-18 +0000,cbc,"New York Times, AP, Reuters and Newsmax among ...",https://www.cbc.ca/news/world/new-york-times-a...
121,53,2025-10-14-14-41-00 +0000,wsj,Maine’s Democratic Gov. Janet Mills officially...,https://www.wsj.com/politics/elections/susan-c...
166,52,2025-10-14-12-17-51 +0000,nypost,Hamas carries out public executions — just hou...,https://nypost.com/2025/10/14/world-news/hamas...
154,52,2025-10-14-13-00-00 +0000,wsj,"Tariffs, inflation and other threats such as e...",https://www.wsj.com/economy/trade-tensions-wei...
266,43,2025-10-14-01-00-00 +0000,wsj,"The White House move to lay off roughly 4,000 ...",https://www.wsj.com/politics/policy/trump-gove...
129,40,2025-10-14-14-20-00 +0000,wsj,China’s Rare-Earth Export Controls Are ‘Critic...,https://www.wsj.com/economy/trade/chinas-rare-...
54,38,2025-10-14-17-12-21 +0000,bbc,US Supreme Court rejects Alex Jones appeal in ...,https://www.bbc.com/news/articles/c5ypjjxd40qo...
273,37,2025-10-14-00-28-00 +0000,wsj,Salesforce’s $15 Billion Investment in San Fra...,https://www.wsj.com/tech/ai/salesforces-15-bil...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
